Importing Dependencies

In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
GEOJSON_DIR_PATH = "../../data/geojson-data"
FINAL_PLOT_DATA = "../../data/field-data/final.csv"

In [3]:

if not os.path.exists(GEOJSON_DIR_PATH):
    os.makedirs(GEOJSON_DIR_PATH)

Loading our data

In [4]:
df = pd.read_csv(FINAL_PLOT_DATA)
rows, _ = df.shape
print(df.head())
print('\nNumber of plots = ', rows)

    plot_lon   plot_lat  plot_agb_mean_tha  subplot_count  plotId
0  80.414795  28.870564         477.230220              2   10-75
1  80.405569  29.192646         367.626079              1   10-84
2  80.394871  29.483126          45.633926              3   10-92
3  84.104420  28.228486         135.266012              4  100-56
4  84.145391  27.650959         329.968995              6  101-40

Number of plots =  524


*To create individual GeoJSON files for each of our 524 plots, we will iterate through our final plot dataset and create a circular AOI for each, and save them into a dedicated directory. We will use UTM Zone 45N (EPSG:32645) for the metric calculations (meters) to ensure our $750m^2$ or $500m^2$ areas are exact, then convert them back to WGS84 (EPSG:4326) for the final GeoJSON files.*

In [5]:
# Convert your pandas DataFrame to a GeoDataFrame (Points)
gdf_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.plot_lon, df.plot_lat),
    crs="EPSG:4326",
)

In [6]:
# Project to a metric CRS (UTM 45N) to create accurate meter-based buffers
gdf_metric = gdf_points.to_crs(epsg=32645)
# Define Radius (Example: 15.45m for a 750m2 plot)
radius_val = np.sqrt(750 / np.pi)

Extracting the geojson data of our plots

In [7]:
# Iterate through each row and save as a separate GeoJSON
print(f"Exporting {len(gdf_metric)} files to {GEOJSON_DIR_PATH}...")
for index, row in gdf_metric.iterrows():
    # Create the buffer (Polygon) for this specific row
    aoi_polygon = row.geometry.buffer(radius_val)

    # Create a single-row GeoDataFrame for export
    single_aoi = gpd.GeoDataFrame([row], geometry=[aoi_polygon], crs=gdf_metric.crs)

    # Project back to WGS84 for the GeoJSON standard
    single_aoi_wgs84 = single_aoi.to_crs(epsg=4326)

    # Clean file name using the unique_plot_id we created earlier
    file_name = f"plot-{row['plotId']}.geojson"
    file_path = os.path.join(GEOJSON_DIR_PATH, file_name)

    # Save to file
    single_aoi_wgs84.to_file(file_path, driver="GeoJSON")

print("Done! All records are saved in the 'geojson-data' directory.")

Exporting 524 files to ../../data/geojson-data...
Done! All records are saved in the 'geojson-data' directory.
